#### Install Package

In [1]:
!pip install sastrawi scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 3.0 MB/s eta 0:00:00


#### Import Library

In [2]:
import pandas as pd
pd.options.mode.chained_assignment = None
from google.colab import files
import re
import string
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import csv
import requests
from io import StringIO
from gensim.models import Word2Vec
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


#### Upload Dataset

In [3]:
uploaded = files.upload()

Saving review_tokopedia.csv to review_tokopedia.csv


#### Load Dataset

In [4]:
df = pd.read_csv('review_tokopedia.csv')
df.head()

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion
0,913b4a1e-32ac-4331-80bf-293e40e01340,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Berkali-kali pengiriman bermasalah. Paket seri...,1,87,3.284.0,2024-10-10 09:23:06,"Halo Toppers, terima kasih atas rating yang ka...",2024-10-10 09:40:27,3.284.0
1,6bd5bd88-f16f-4a3c-9826-d31dbd713062,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,menu kupon sering hilang dan sangat- sangat su...,1,87,3.283.0,2024-10-03 21:59:24,"Hi Toppers, silakan sampaikan kritik/saran ata...",2024-10-03 22:20:04,3.283.0
2,97a613e7-23e6-4c09-994e-13c290062fed,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"Saya beli kebutuhan anak di Tokopedia, tetapi ...",1,66,3.283.0,2024-10-08 02:27:52,"Hi Toppers, silakan sampaikan kritik/saran ata...",2024-10-08 03:21:20,3.283.0
3,df920305-fa08-430a-9ab0-39df61468c56,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Tolong perbaiki kurir rekomendasi dan layanan ...,2,40,3.282.0,2024-10-02 03:12:31,"Hi Toppers, silakan sampaikan kritik/saran ata...",2024-10-02 03:40:10,3.282.0
4,77fb02de-e2d9-44fc-8796-cbc9134cc1c7,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Ternyata tiket promo hilang jika sudah masuk m...,1,41,3.282.0,2024-10-01 05:17:16,"Toppers, mohon maaf atas kendala yang kamu ala...",2024-10-01 05:20:14,3.282.0


#### Display Data Frame Information

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121500 entries, 0 to 121499
Data columns (total 11 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   reviewId              121500 non-null  object
 1   userName              121500 non-null  object
 2   userImage             121500 non-null  object
 3   content               121500 non-null  object
 4   score                 121500 non-null  int64 
 5   thumbsUpCount         121500 non-null  int64 
 6   reviewCreatedVersion  86419 non-null   object
 7   at                    121500 non-null  object
 8   replyContent          66225 non-null   object
 9   repliedAt             66225 non-null   object
 10  appVersion            86419 non-null   object
dtypes: int64(2), object(9)
memory usage: 10.2+ MB


#### Clean Data Frame

In [6]:
clean_df = df.dropna()
clean_df = clean_df.drop_duplicates()
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 52110 entries, 0 to 121497
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   reviewId              52110 non-null  object
 1   userName              52110 non-null  object
 2   userImage             52110 non-null  object
 3   content               52110 non-null  object
 4   score                 52110 non-null  int64 
 5   thumbsUpCount         52110 non-null  int64 
 6   reviewCreatedVersion  52110 non-null  object
 7   at                    52110 non-null  object
 8   replyContent          52110 non-null  object
 9   repliedAt             52110 non-null  object
 10  appVersion            52110 non-null  object
dtypes: int64(2), object(9)
memory usage: 4.8+ MB


#### Indonesian Slangwords

In [7]:
slangwords = {
    "@": "di",
    "abis": "habis",
    "wtb": "beli",
    "masi": "masih",
    "wts": "jual",
    "wtt": "tukar",
    "bgt": "banget",
    "maks": "maksimal",
    "jg": "juga",
    "udh": "sudah",
    "blm": "belum",
    "trs": "terus",
    "sm": "sama",
    "kalo": "kalau",
    "pdhl": "padahal",
    "tp": "tapi",
    "btw": "omong-omong",
    "sy": "saya",
    "aq": "aku",
    "km": "kamu",
    "kmu": "kamu",
    "gpp": "tidak apa-apa",
    "gak": "tidak",
    "ga": "tidak",
    "tdk": "tidak",
    "bs": "bisa",
    "msh": "masih",
    "bbrp": "beberapa",
    "pngn": "pengen",
    "inget": "ingat",
    "bnyk": "banyak",
    "sdh": "sudah",
    "org": "orang",
    "hrs": "harus",
    "prnh": "pernah",
    "brg": "barang",
    "sblm": "sebelum",
    "ss": "screenshoot",
    "sdg": "sedang",
    "dpt": "dapat",
    "bkn": "bukan",
    "tpi": "tapi",
    "bsa": "bisa",
    "kyk": "kayak",
    "pd": "pada",
    "gtu": "gitu",
    "kek": "seperti",
    "dr": "dari",
    "dgn": "dengan",
    "ntar": "nanti",
    "gmn": "bagaimana",
    "dlu": "dulu",
    "smua": "semua",
    "smg": "semoga",
    "smga": "semoga",
    "jd": "jadi",
    "yg": "yang",
    "kyknya": "kayaknya",
    "klo": "kalau",
    "mnrt": "menurut",
    "brp": "berapa",
    "sbnrnya": "sebenarnya",
    "aj": "aja",
    "aja": "saja",
    "bgtu": "begitu",
    "kmrn": "kemarin",
    "sblum": "sebelum",
    "krn": "karena",
    "knp": "kenapa",
    "bngt": "banget",
    "td": "tadi",
    "gimana": "bagaimana",
    "mungkin": "mungkin",
    "mngkin": "mungkin",
    "tlg": "tolong",
    "tol": "tolong"
}

#### Text Preprocess Functions

In [8]:
def cleaning_text(text):
  text = re.sub(r'@[A-Za-z0-9]+', '', text)
  text = re.sub(r'#[A-Za-z0-9]+', '', text)
  text = re.sub(r'RT[\s]', '', text)
  text = re.sub(r"http\S+", '', text)
  text = re.sub(r'[0-9]+', '', text)
  text = re.sub(r'[^\w\s]', '', text)

  text = text.replace('\n', ' ')
  text = text.strip(' ')
  return text

def casefolding_text(text):
  return text.lower()

def tokenize_text(text):
  return word_tokenize(text)

def filter_text(text):
  stop_words = set(stopwords.words('indonesian'))
  stop_words_en = set(stopwords.words('english'))
  stop_words.update(stop_words_en)
  stop_words.update(['iya','yaa','gak','nya','na','sih','ku',"di","ga","ya","gaa","loh","kah","woi","woii","woy"])
  filtered = []
  for word in text:
    if word not in stop_words:
      filtered.append(word)
  return filtered

def stemming_text(text):
  factory = StemmerFactory()
  stemmer = factory.create_stemmer()
  words = text.split()
  stemmed_words = [stemmer.stem(word) for word in words]
  stemmed_text = ' '.join(stemmed_words)
  return stemmed_text

def to_sentence(words):
  return ' '.join(word for word in words)

def fix_slangwords(text):
  words = text.split()
  corrected_words = []

  for word in words:
    if word.lower() in slangwords:
      corrected_words.append(slangwords[word.lower()])
    else:
      corrected_words.append(word)

  return ' '.join(corrected_words)

#### Text Preprocess

In [9]:
clean_df['text_clean'] = clean_df['content'].apply(cleaning_text)
clean_df['text_casefoldingText'] = clean_df['text_clean'].apply(casefolding_text)
clean_df['text_slangwords'] = clean_df['text_casefoldingText'].apply(fix_slangwords)
clean_df['text_tokenizingText'] = clean_df['text_slangwords'].apply(tokenize_text)
clean_df['text_stopword'] = clean_df['text_tokenizingText'].apply(filter_text)
clean_df['text_final'] = clean_df['text_stopword'].apply(to_sentence)

#### Fetch Positive and Negative Words Dictionary

In [10]:
lexicon_positive = dict()

response = requests.get('https://raw.githubusercontent.com/angelmetanosaa/dataset/main/lexicon_positive.csv')

if response.status_code == 200:
    reader = csv.reader(StringIO(response.text), delimiter=',')
    for row in reader:
        lexicon_positive[row[0]] = int(row[1])
else:
    print("Failed to fetch positive lexicon data")

lexicon_negative = dict()

response = requests.get('https://raw.githubusercontent.com/angelmetanosaa/dataset/main/lexicon_negative.csv')

if response.status_code == 200:
    reader = csv.reader(StringIO(response.text), delimiter=',')
    for row in reader:
        lexicon_negative[row[0]] = int(row[1])
else:
    print("Failed to fetch negative lexicon data")

#### Sentiment Analysis Function

In [11]:
def sentiment_analysis(text):
    score = 0

    for word in text:
        if word in lexicon_positive:
            score += lexicon_positive[word]

    for word in text:
        if word in lexicon_negative:
            score += lexicon_negative[word]

    if score > 0:
        polarity = 'positive'
    elif score < 0:
        polarity = 'negative'
    else:
        polarity = 'neutral'

    return score, polarity

In [12]:
results = clean_df['text_stopword'].apply(sentiment_analysis)
results = list(zip(*results))
clean_df['polarity_score'] = results[0]
clean_df['polarity'] = results[1]
print(clean_df['polarity'].value_counts())

polarity
negative    25106
positive    23041
neutral      3963
Name: count, dtype: int64


In [13]:
print(clean_df['polarity'].value_counts())

from sklearn.utils import resample

df_majority = clean_df[clean_df['polarity'] == 'positive']
df_minority_neg = clean_df[clean_df['polarity'] == 'negative']
df_minority_neu = clean_df[clean_df['polarity'] == 'neutral']

df_minority_neg_upsampled = resample(df_minority_neg,
                                     replace=True,
                                     n_samples=len(df_majority),
                                     random_state=42)

df_minority_neu_upsampled = resample(df_minority_neu,
                                     replace=True,
                                     n_samples=len(df_majority),
                                     random_state=42)

clean_df = pd.concat([df_majority, df_minority_neg_upsampled, df_minority_neu_upsampled])
print(clean_df['polarity'].value_counts())

polarity
negative    25106
positive    23041
neutral      3963
Name: count, dtype: int64
polarity
positive    23041
negative    23041
neutral     23041
Name: count, dtype: int64


#### Data Extraction: TF-IDF

In [14]:
X = clean_df['text_final']
y = clean_df['polarity']

tfidf = TfidfVectorizer(max_features=5000, min_df=5, max_df=0.8, ngram_range=(1,3))
X_tfidf = tfidf.fit_transform(X)

tfidf_df = pd.DataFrame(X_tfidf.toarray(), columns=tfidf.get_feature_names_out())

tfidf_df

,aamiin,abal,abal abal,academy,academy tokopedia,acak,acara,acc,account,ad,...,yah,yak,yaudah,yes,yh,yng,youtube,yuk,zonk,𝚍𝚒
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69118,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
69119,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
69120,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
69121,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### Data Extraction: Word2Vec

In [15]:
X2 = clean_df['text_final'].apply(lambda x: x.split())

w2v_model = Word2Vec(sentences=X2, vector_size=100, window=5, min_count=2, workers=4)

def document_vector(w2v_model, doc):
    """Create document vectors by averaging the word vectors in the document"""
    doc = [word for word in doc if word in w2v_model.wv]
    if len(doc) == 0:
        return np.zeros(w2v_model.vector_size)
    return np.mean([w2v_model.wv[word] for word in doc], axis=0)

X_w2v = np.array([document_vector(w2v_model, doc) for doc in X2])

w2v_df = pd.DataFrame(X_w2v)

w2v_df

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-0.063927,-0.636297,-0.099218,-0.688405,0.823470,-0.305476,0.098459,0.748111,-0.034193,-0.554027,...,0.484592,1.110360,-0.268665,0.823559,0.214173,0.853701,-0.028439,-0.099232,0.064662,-0.216136
1,-0.121037,-0.031964,-0.556741,-0.509691,0.655911,-1.159824,0.295768,0.470026,-0.277559,-0.652379,...,0.317839,0.241479,0.302975,-0.351178,-0.061674,0.297728,0.353609,-0.149787,-0.233649,-0.287792
2,0.143801,-0.315911,-0.239673,-1.316842,1.121001,-0.185827,0.273253,0.628694,-0.189482,-0.835326,...,1.159226,0.820863,-0.189906,0.272286,0.200663,1.119142,-0.491017,-0.413418,0.414578,-0.525255
3,-0.330025,-0.277511,-0.324629,-0.090230,0.446017,-1.308735,0.444576,0.372881,0.276406,-0.703918,...,0.127638,0.155535,0.127243,-0.115626,0.225590,0.336459,-0.130160,0.020378,-0.509268,0.191468
4,-0.006126,-0.500075,0.009575,-0.946967,0.756846,0.170732,0.034853,0.320127,-0.070724,-0.755880,...,0.458345,0.828129,-0.217379,0.619562,0.326609,0.753767,-0.068787,-0.035263,0.314959,-0.300349
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69118,-0.686282,0.502663,-0.798266,0.889337,0.193533,-1.924417,1.213384,1.751023,1.436707,0.398239,...,1.140809,-0.923840,1.784144,-1.295211,0.488780,0.826921,1.265240,0.585971,0.799931,-0.616879
69119,-0.041438,-0.043121,-0.388304,-0.519025,0.275780,-1.083725,0.193160,0.412074,-0.257519,-0.856858,...,0.995993,0.087777,0.083535,0.052295,0.479087,1.281695,-0.088624,-0.468300,-0.049344,-0.765618
69120,-0.243700,-0.235799,-0.167279,-0.118385,0.301238,-0.996318,0.270687,0.928421,0.041603,-0.469845,...,0.030994,0.666326,0.112084,0.440617,0.425629,0.081316,0.072551,-0.480974,-0.028156,0.262540
69121,-0.312969,-0.624126,-1.081650,0.076886,0.528087,-2.062709,1.034512,0.554183,0.076367,-0.403207,...,0.170282,-0.603038,-0.021955,0.430152,-0.053206,-0.082434,0.070784,-0.279200,-1.019881,0.676558


#### Split Dataset

In [16]:
# TF-IDF 80/20
tfidf_X_train1, tfidf_X_test1, tfidf_y_train1, tfidf_y_test1 = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

# Word2Vec 80/20
w2v_X_train, w2v_X_test, w2v_y_train, w2v_y_test = train_test_split(X_w2v, y, test_size=0.2, random_state=42)

# TF-IDF 70/30
tfidf_X_train2, tfidf_X_test2, tfidf_y_train2, tfidf_y_test2 = train_test_split(X_tfidf, y, test_size=0.3, random_state=42)

#### SVM | TF-IDF | 80/20

In [17]:
svm_model = SVC(kernel='linear')
svm_model.fit(tfidf_X_train1, tfidf_y_train1)

y_pred_train_svm = svm_model.predict(tfidf_X_train1)
accuracy_train_svm = accuracy_score(tfidf_y_train1, y_pred_train_svm)

y_pred_test_svm = svm_model.predict(tfidf_X_test1)
accuracy_test_svm = accuracy_score(tfidf_y_test1, y_pred_test_svm)

print('Support Vector Machine - TF-IDF 80/20')
print(f'Accuracy Train: {accuracy_train_svm * 100:.2f}%')
print(f'Accuracy Test: {accuracy_test_svm * 100:.2f}%')
print(classification_report(tfidf_y_test1, y_pred_test_svm))

Support Vector Machine - TF-IDF 80/20
Accuracy Train: 91.80%
Accuracy Test: 88.14%
              precision    recall  f1-score   support

    negative       0.92      0.87      0.90      4618
     neutral       0.80      0.93      0.86      4648
    positive       0.94      0.84      0.89      4559

    accuracy                           0.88     13825
   macro avg       0.89      0.88      0.88     13825
weighted avg       0.89      0.88      0.88     13825



Random Forest | TF-IDF | 70/30

In [18]:
random_forest_tfidf = RandomForestClassifier(n_estimators=100, random_state=42)
random_forest_tfidf.fit(tfidf_X_train2, tfidf_y_train2)

y_pred_train_rf = random_forest_tfidf.predict(tfidf_X_train2)
accuracy_train_rf = accuracy_score(tfidf_y_train2, y_pred_train_rf)

y_pred_test_rf = random_forest_tfidf.predict(tfidf_X_test2)
accuracy_test_rf = accuracy_score(tfidf_y_test2, y_pred_test_rf)

print('Random Forest - TF-IDF 70/30')
print(f'Accuracy Train: {accuracy_train_rf * 100:.2f}%')
print(f'Accuracy Test: {accuracy_test_rf * 100:.2f}%')
print(classification_report(tfidf_y_test2, y_pred_test_rf))

Random Forest - TF-IDF 70/30
Accuracy Train: 99.95%
Accuracy Test: 91.64%
              precision    recall  f1-score   support

    negative       0.88      0.90      0.89      6944
     neutral       0.96      0.99      0.98      6949
    positive       0.90      0.86      0.88      6844

    accuracy                           0.92     20737
   macro avg       0.92      0.92      0.92     20737
weighted avg       0.92      0.92      0.92     20737



#### Random Forest | Word2Vec | 80/20

In [19]:
random_forest_w2v = RandomForestClassifier(n_estimators=100, random_state=42)
random_forest_w2v.fit(w2v_X_train, w2v_y_train)

y_pred_train_rf = random_forest_w2v.predict(w2v_X_train)
accuracy_train_rf = accuracy_score(w2v_y_train, y_pred_train_rf)

y_pred_test_rf = random_forest_w2v.predict(w2v_X_test)
accuracy_test_rf = accuracy_score(w2v_y_test, y_pred_test_rf)

print('Random Forest - Word2Vec 80/20')
print(f'Accuracy Train: {accuracy_train_rf * 100:.2f}%')
print(f'Accuracy Test: {accuracy_test_rf * 100:.2f}%')
print(classification_report(w2v_y_test, y_pred_test_rf))

Random Forest - Word2Vec 80/20
Accuracy Train: 100.00%
Accuracy Test: 88.56%
              precision    recall  f1-score   support

    negative       0.82      0.85      0.84      4618
     neutral       0.99      1.00      0.99      4648
    positive       0.85      0.81      0.83      4559

    accuracy                           0.89     13825
   macro avg       0.89      0.88      0.88     13825
weighted avg       0.89      0.89      0.89     13825



#### Inference

In [20]:
# SVM TF-IDF
def predict_svm(text):
    cleaned = cleaning_text(text)
    cleaned = casefolding_text(cleaned)
    cleaned = fix_slangwords(cleaned)
    tokens = tokenize_text(cleaned)
    filtered = filter_text(tokens)
    final_text = to_sentence(filtered)
    vector = tfidf.transform([final_text])
    prediction = svm_model.predict(vector)
    return prediction[0]

# RF TF-IDF
def predict_rf_tfidf(text):
    cleaned = cleaning_text(text)
    cleaned = casefolding_text(cleaned)
    cleaned = fix_slangwords(cleaned)
    tokens = tokenize_text(cleaned)
    filtered = filter_text(tokens)
    final_text = to_sentence(filtered)
    vector = tfidf.transform([final_text])
    prediction = random_forest_tfidf.predict(vector)
    return prediction[0]

# RF Word2Vec
def predict_rf_word2vec(text):
    cleaned = cleaning_text(text)
    cleaned = casefolding_text(cleaned)
    cleaned = fix_slangwords(cleaned)
    tokens = tokenize_text(cleaned)
    filtered = filter_text(tokens)
    final_text = to_sentence(filtered)
    tokens = final_text.split()
    vector = document_vector(w2v_model, tokens).reshape(1, -1)
    prediction = random_forest_w2v.predict(vector)
    return prediction[0]

In [28]:
sample_text = "Produk ini sangat bagus dan berkualitas tinggi!"

# SVM TF-IDF
svm_prediction = predict_svm(sample_text)
print(f"SVM Prediction: {svm_prediction}")

# Random Forest TF-IDF
rf_tfidf_prediction = predict_rf_tfidf(sample_text)
print(f"Random Forest Prediction (TF-IDF): {rf_tfidf_prediction}")

# Random Forest Word2Vec
rf_word2vec_prediction = predict_rf_word2vec(sample_text)
print(f"Random Forest Prediction (Word2Vec): {rf_word2vec_prediction}")

SVM Prediction: positive
Random Forest Prediction (TF-IDF): positive
Random Forest Prediction (Word2Vec): positive


#### Get Requirement

In [22]:
!pip freeze > requirements.txt